# Introduction

# 1. Import Modules

In [1]:
import os

# Import the pandas module
import pandas

# Import the geopandas module
import geopandas

# Import the rsgislib classification module
import rsgislib.classification

# Import rsgislib vectorutils module 
import rsgislib.vectorutils

# Import the rsgislib tools utils module
import rsgislib.tools.utils

# Import the function to calculate the accuracy stats
from rsgislib.classification.classaccuracymetrics import calc_acc_metrics_vecsamples

# 2. Create Output Directories

In [2]:
out_pts_dir = "acc_pts"
if not os.path.exists(out_pts_dir):
    os.mkdir(out_pts_dir)
    
out_stats_dir = "acc_stats"
if not os.path.exists(out_stats_dir):
    os.mkdir(out_stats_dir)

# 3. Define Input Files

In [3]:
refl_cls_dir = "cls_refl_results"
refl_cls_ml_img = os.path.join(refl_cls_dir, "cls_ml_refl.kea")
refl_cls_svm_img = os.path.join(refl_cls_dir, "cls_svm_refl.kea")
refl_cls_rf_img = os.path.join(refl_cls_dir, "cls_rf_refl.kea")
refl_cls_et_img = os.path.join(refl_cls_dir, "cls_et_refl.kea")
refl_cls_gbt_img = os.path.join(refl_cls_dir, "cls_gbt_refl.kea")
refl_cls_nn_img = os.path.join(refl_cls_dir, "cls_nn_refl.kea")

linnorm_cls_dir = "cls_lin_norm_results"
linnorm_cls_ml_img = os.path.join(linnorm_cls_dir, "cls_ml_linnorm.kea")
linnorm_cls_svm_img = os.path.join(linnorm_cls_dir, "cls_svm_linnorm.kea")
linnorm_cls_rf_img = os.path.join(linnorm_cls_dir, "cls_rf_linnorm.kea")
linnorm_cls_et_img = os.path.join(linnorm_cls_dir, "cls_et_linnorm.kea")
linnorm_cls_gbt_img = os.path.join(linnorm_cls_dir, "cls_gbt_linnorm.kea")
linnorm_cls_nn_img = os.path.join(linnorm_cls_dir, "cls_nn_linnorm.kea")

sdnorm_cls_dir = "cls_sdnorm_results"
sdnorm_cls_ml_img = os.path.join(sdnorm_cls_dir, "cls_ml_sdnorm.kea")
sdnorm_cls_svm_img = os.path.join(sdnorm_cls_dir, "cls_svm_sdnorm.kea")
sdnorm_cls_rf_img = os.path.join(sdnorm_cls_dir, "cls_rf_sdnorm.kea")
sdnorm_cls_et_img = os.path.join(sdnorm_cls_dir, "cls_et_sdnorm.kea")
sdnorm_cls_gbt_img = os.path.join(sdnorm_cls_dir, "cls_gbt_sdnorm.kea")
sdnorm_cls_nn_img = os.path.join(sdnorm_cls_dir, "cls_nn_sdnorm.kea")

vec_file = "../data/cls_data/cls_acc_assessment_pts_ref.geojson"
vec_lyr = "cls_acc_assessment_pts_ref"

# 4. Copy Existing Points File

To avoid overwriting and editting the input file provided we will first copy it into our output directory.

In [4]:
vec_refpts_file = os.path.join(out_pts_dir, "cls_acc_assessment_pts_compare_ref.geojson")
vec_refpts_lyr = "cls_acc_assessment_pts_compare_ref"

rsgislib.vectorutils.vector_translate(vec_file, vec_lyr, vec_refpts_file, vec_refpts_lyr, out_format="GeoJSON", del_exist_vec=True)

Deleting: acc_pts/cls_acc_assessment_pts_compare_ref.geojson


 10%|███████████████████▏                                                                                                                                                                          | 101/1000 [00:00<00:00, 8855.77it/s]


# 5. Create `dict` look up table (LUT)

To reduce the amount of code we need to write, reducing duplication and improving code relability is often better to use a loop with a look up table (LUT) for the input and output parameters. In this case we will use a `dict` as provide that LUT. 


In [5]:
cls_info = dict()
cls_info["ml_rl_cls"] = refl_cls_ml_img
cls_info["svm_rl_cls"] = refl_cls_svm_img
cls_info["rf_rl_cls"] = refl_cls_rf_img
cls_info["et_rl_cls"] = refl_cls_et_img
cls_info["gbt_rl_cls"] = refl_cls_gbt_img
cls_info["nn_rl_cls"] = refl_cls_nn_img

cls_info["ml_nln_cls"] = linnorm_cls_ml_img
cls_info["svm_nln_cls"] = linnorm_cls_svm_img
cls_info["rf_nln_cls"] = linnorm_cls_rf_img
cls_info["et_nln_cls"] = linnorm_cls_et_img
cls_info["gbt_nln_cls"] = linnorm_cls_gbt_img
cls_info["nn_nln_cls"] = linnorm_cls_nn_img

cls_info["ml_nsd_cls"] = sdnorm_cls_ml_img
cls_info["svm_nsd_cls"] = sdnorm_cls_svm_img
cls_info["rf_nsd_cls"] = sdnorm_cls_rf_img
cls_info["et_nsd_cls"] = sdnorm_cls_et_img
cls_info["gbt_nsd_cls"] = sdnorm_cls_gbt_img
cls_info["nn_nsd_cls"] = sdnorm_cls_nn_img

# 6. Populate Accuracy Reference Points

In [6]:
for cls_col in cls_info:
    print(cls_col)
    rsgislib.classification.pop_class_info_accuracy_pts(
        input_img=cls_info[cls_col],
        vec_file=vec_refpts_file,
        vec_lyr=vec_refpts_lyr,
        rat_class_col="class_names",
        vec_class_col=cls_col,
        vec_ref_col=None,
        vec_process_col=None,
    )
cls_cols = list(cls_info.keys())

ml_rl_cls

svm_rl_cls
rf_rl_cls

et_rl_cls


gbt_rl_cls

nn_rl_cls

ml_nln_cls
svm_nln_cls

rf_nln_cls


et_nln_cls
gbt_nln_cls

nn_nln_cls
ml_nsd_cls



svm_nsd_cls

rf_nsd_cls
et_nsd_cls

gbt_nsd_cls

nn_nsd_cls




# 7. Filter Valid Points

Some classifiers can produce no data regions and if those intersect with reference points then an error will occur when calculating the accuracy statistics so we need to remove those which we will do using geopandas. For points where there is no class (i.e., no data) in the input classification then the value `"NA"` is outputted into the attribute table and it is rows with an `"NA"` value that we want to remove.


In [7]:
vec_refpts_vld_file = os.path.join(out_pts_dir, "cls_acc_assessment_pts_compare_ref_vld.geojson")
vec_refpts_vld_lyr = "cls_acc_assessment_pts_compare_ref_vld"
points_gdf = geopandas.read_file(vec_refpts_file)
for cls_col in cls_cols:
    print(cls_col)
    points_gdf = points_gdf.drop(points_gdf[points_gdf[cls_col] == "NA"].index)

points_gdf.to_file(vec_refpts_vld_file, driver="GeoJSON")
points_gdf

ml_rl_cls
svm_rl_cls
rf_rl_cls
et_rl_cls
gbt_rl_cls
nn_rl_cls
ml_nln_cls
svm_nln_cls
rf_nln_cls
et_nln_cls
gbt_nln_cls
nn_nln_cls
ml_nsd_cls
svm_nsd_cls
rf_nsd_cls
et_nsd_cls
gbt_nsd_cls
nn_nsd_cls


/Users/pete/miniforge3/envs/osgeo-env-v2/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


,ref_pts,ml_rl_cls,svm_rl_cls,rf_rl_cls,et_rl_cls,gbt_rl_cls,nn_rl_cls,ml_nln_cls,svm_nln_cls,rf_nln_cls,et_nln_cls,gbt_nln_cls,nn_nln_cls,ml_nsd_cls,svm_nsd_cls,rf_nsd_cls,et_nsd_cls,gbt_nsd_cls,nn_nsd_cls,geometry
0,Bare_Rock_Sand,Artificial_Surfaces,NonPhoto_Veg,Artificial_Surfaces,Artificial_Surfaces,Grass_Short,NonPhoto_Veg,Artificial_Surfaces,NonPhoto_Veg,Artificial_Surfaces,Artificial_Surfaces,Grass_Short,NonPhoto_Veg,Bare_Rock_Sand,Bare_Rock_Sand,Bare_Rock_Sand,Artificial_Surfaces,Grass_Short,Artificial_Surfaces,POINT (293492.332 306597.255)
1,Artificial_Surfaces,Grass_Short,Artificial_Surfaces,Grass_Short,Grass_Short,Artificial_Surfaces,Artificial_Surfaces,Grass_Short,Grass_Short,Grass_Short,Grass_Short,Artificial_Surfaces,Grass_Short,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,POINT (312462.332 304967.255)
2,Bare_Rock_Sand,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,Bare_Rock_Sand,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,Bare_Rock_Sand,Grass_Short,Bare_Rock_Sand,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,NonPhoto_Veg,POINT (306412.332 293567.255)
3,Artificial_Surfaces,Grass_Short,Artificial_Surfaces,Scrub,Scrub,Artificial_Surfaces,Artificial_Surfaces,Grass_Short,Bare_Rock_Sand,Scrub,Scrub,Artificial_Surfaces,Grass_Short,Grass_Short,Artificial_Surfaces,Scrub,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,POINT (259012.332 309877.255)
4,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,Artificial_Surfaces,POINT (305842.332 323467.255)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,POINT (250902.332 316807.255)
996,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,POINT (257912.332 297147.255)
997,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,POINT (249762.332 311677.255)
998,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,Water,POINT (253712.332 295707.255)


# 8. Calculate Classification Accuracy Stats

An LUT of the classification accuracy statistics JSON files will also be created to allow post processing and summarising of the classification accuracy statistics.


In [8]:
cls_acc_stats_lut = dict()
for cls_col in cls_info:
    print(cls_col)
    out_json_file = os.path.join(out_stats_dir, f"{cls_col}_acc_info.json")
    out_csv_file = os.path.join(out_stats_dir, f"{cls_col}_acc_info.csv")

    calc_acc_metrics_vecsamples(
        vec_file=vec_refpts_vld_file,
        vec_lyr=vec_refpts_vld_lyr,
        ref_col="ref_pts",
        cls_col=cls_col,
        cls_img=cls_info[cls_col],
        img_cls_name_col="class_names",
        img_hist_col="Histogram",
        out_json_file=out_json_file,
        out_csv_file=out_csv_file,
    )
    cls_acc_stats_lut[cls_col] = out_json_file

ml_rl_cls
svm_rl_cls
rf_rl_cls
et_rl_cls
gbt_rl_cls
nn_rl_cls
ml_nln_cls
svm_nln_cls
rf_nln_cls
et_nln_cls
gbt_nln_cls
nn_nln_cls
ml_nsd_cls
svm_nsd_cls
rf_nsd_cls
et_nsd_cls
gbt_nsd_cls
nn_nsd_cls


# 9. Summarise Classification Statistics

In [9]:
# A convinent way of creating a pandas dataframe (i.e., table of data)
# is through a dict where the dict keys provide the column names.
cls_acc_stats = dict()
cls_acc_stats["Classifier"] = list()
cls_acc_stats["Proportion Correct"] = list()
cls_acc_stats["Allocation Disagreement"] = list()
cls_acc_stats["Quantity Disagreement"] = list()
cls_acc_stats["Overall Accuracy"] = list()
cls_acc_stats["Kappa"] = list()
cls_acc_stats["macro f1-score"] = list()
cls_acc_stats["weighted area f1-score"] = list()

for cls_col in cls_acc_stats_lut:
    print(cls_col)
    cls_acc_stats_dict = rsgislib.tools.utils.read_json_to_dict(cls_acc_stats_lut[cls_col])
    cls_acc_stats["Classifier"].append(cls_col)
    cls_acc_stats["Proportion Correct"].append(cls_acc_stats_dict["quantity_metrics"]["Proportion Correct (C)"])
    cls_acc_stats["Allocation Disagreement"].append(cls_acc_stats_dict["quantity_metrics"]["Allocation Disagreement (A)"])
    cls_acc_stats["Quantity Disagreement"].append(cls_acc_stats_dict["quantity_metrics"]["Quantity Disagreement (Q)"])
    cls_acc_stats["Overall Accuracy"].append(cls_acc_stats_dict["accuracy"])
    cls_acc_stats["Kappa"].append(cls_acc_stats_dict["cohen_kappa"])
    cls_acc_stats["macro f1-score"].append(cls_acc_stats_dict["macro avg"]["f1-score"])
    cls_acc_stats["weighted area f1-score"].append(cls_acc_stats_dict["weighted area avg"]["f1-score"])
    
    
cls_acc_stats_df = pandas.DataFrame.from_dict(cls_acc_stats)
cls_acc_stats_df.set_index("Classifier")

ml_rl_cls
svm_rl_cls
rf_rl_cls
et_rl_cls
gbt_rl_cls
nn_rl_cls
ml_nln_cls
svm_nln_cls
rf_nln_cls
et_nln_cls
gbt_nln_cls
nn_nln_cls
ml_nsd_cls
svm_nsd_cls
rf_nsd_cls
et_nsd_cls
gbt_nsd_cls
nn_nsd_cls


,Proportion Correct,Allocation Disagreement,Quantity Disagreement,Overall Accuracy,Kappa,macro f1-score,weighted area f1-score
Classifier,,,,,,,
ml_rl_cls,0.793741,0.111214,0.095044,0.677678,0.637514,0.637116,0.703867
svm_rl_cls,0.745326,0.149248,0.105426,0.652653,0.616199,0.641125,0.642229
rf_rl_cls,0.727509,0.209638,0.062853,0.708709,0.675284,0.702466,0.707992
et_rl_cls,0.764514,0.153481,0.082005,0.716717,0.685001,0.709209,0.717549
gbt_rl_cls,0.695808,0.170355,0.133837,0.690691,0.655981,0.691852,0.663728
nn_rl_cls,0.595080,0.299870,0.105049,0.588589,0.544928,0.583740,0.531490
ml_nln_cls,0.792971,0.112475,0.094554,0.676677,0.636421,0.636308,0.702901
svm_nln_cls,0.747794,0.143823,0.108383,0.654655,0.618427,0.642209,0.648508
rf_nln_cls,0.719001,0.217400,0.063599,0.700701,0.665933,0.696331,0.696684


# 10. Sort Summarised Results

In [10]:
cls_acc_stats_sort_df = cls_acc_stats_df.sort_values(by=['Proportion Correct'], ascending=False).set_index("Classifier")
cls_acc_stats_sort_df

,Proportion Correct,Allocation Disagreement,Quantity Disagreement,Overall Accuracy,Kappa,macro f1-score,weighted area f1-score
Classifier,,,,,,,
nn_nln_cls,0.822484,0.105915,0.071601,0.617618,0.571217,0.590591,0.607266
ml_rl_cls,0.793741,0.111214,0.095044,0.677678,0.637514,0.637116,0.703867
ml_nsd_cls,0.793002,0.109274,0.097723,0.666667,0.625035,0.625287,0.701673
ml_nln_cls,0.792971,0.112475,0.094554,0.676677,0.636421,0.636308,0.702901
et_rl_cls,0.764514,0.153481,0.082005,0.716717,0.685001,0.709209,0.717549
svm_nln_cls,0.747794,0.143823,0.108383,0.654655,0.618427,0.642209,0.648508
et_nln_cls,0.745888,0.156449,0.097663,0.705706,0.672970,0.696996,0.699595
svm_rl_cls,0.745326,0.149248,0.105426,0.652653,0.616199,0.641125,0.642229
et_nsd_cls,0.741136,0.170236,0.088628,0.721722,0.690432,0.715227,0.716180


In [11]:
cls_acc_stats_sort_df = cls_acc_stats_df.sort_values(by=['macro f1-score'], ascending=False).set_index("Classifier")
cls_acc_stats_sort_df

,Proportion Correct,Allocation Disagreement,Quantity Disagreement,Overall Accuracy,Kappa,macro f1-score,weighted area f1-score
Classifier,,,,,,,
et_nsd_cls,0.741136,0.170236,0.088628,0.721722,0.690432,0.715227,0.716180
rf_nsd_cls,0.738171,0.197087,0.064742,0.718719,0.686553,0.713563,0.724373
et_rl_cls,0.764514,0.153481,0.082005,0.716717,0.685001,0.709209,0.717549
rf_rl_cls,0.727509,0.209638,0.062853,0.708709,0.675284,0.702466,0.707992
et_nln_cls,0.745888,0.156449,0.097663,0.705706,0.672970,0.696996,0.699595
rf_nln_cls,0.719001,0.217400,0.063599,0.700701,0.665933,0.696331,0.696684
gbt_rl_cls,0.695808,0.170355,0.133837,0.690691,0.655981,0.691852,0.663728
gbt_nln_cls,0.705551,0.155985,0.138463,0.688689,0.653754,0.689857,0.662587
gbt_nsd_cls,0.690957,0.165876,0.143167,0.675676,0.639770,0.680273,0.647035


# 11. Conclusions

So, what can we interpret from the result above? 

 1. The overall accuracies are must lower than the test/train scores we looked earlier - due to poor sample data as discussed.
 2. The order of the classifiers in terms of accuracy is very different to those using the test/train scores.
 3. The order of the classifiers is very different when using the "Proportion Correct" (areas normalised) verses the "macro f1-score" (not area normalised)
